## 자산_부채총계_데이터전처리

In [1]:
import pandas as pd
import requests
import json
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import time
time.sleep(0.3)
pd.options.display.float_format = '{:.6f}'.format

In [2]:
cdf = pd.read_csv('data/company_name_and_code.csv', index_col = 0)
cdf

,corp_code,corp_name,stock_code
0,365387,AJ네트웍스,95570
1,125080,AK홀딩스,6840
2,219097,BGF,27410
3,1263022,BGF리테일,282330
4,858364,BNK금융지주,138930
...,...,...,...
726,111421,휴니드테크놀러지스,5870
727,362238,휴비스,79980
728,156488,휴스틸,5010
729,103176,흥국화재,540


In [3]:
adf_2019 = pd.read_csv('data/total_asset_debt/total_asset_debt_2019.csv')
adf_2020 = pd.read_csv('data/total_asset_debt/total_asset_debt_2020.csv')
adf_2021 = pd.read_csv('data/total_asset_debt/total_asset_debt_2021.csv')
adf_2022 = pd.read_csv('data/total_asset_debt/total_asset_debt_2022.csv')
adf_2023 = pd.read_csv('data/total_asset_debt/total_asset_debt_2023.csv')
adf_2024 = pd.read_csv('data/total_asset_debt/total_asset_debt_2024.csv')

In [4]:
adf_2019

,corp_code,corp_name,year,total_asset,ln_asset,total_debt
0,365387,AJ네트웍스,2019,1803255225986.000000,28.220615,1455914070366.000000
1,125080,AK홀딩스,2019,4327964555085.000000,29.096118,2898025666994.000000
2,219097,BGF,2019,1657381460248.000000,28.136260,129126735585.000000
3,1263022,BGF리테일,2019,2172410899706.000000,28.406859,1549815137631.000000
4,858364,BNK금융지주,2019,0.000000,0.000000,0.000000
...,...,...,...,...,...,...
726,111421,휴니드테크놀러지스,2019,260965121649.000000,26.287653,145116828409.000000
727,362238,휴비스,2019,877170812678.000000,27.499968,481045043591.000000
728,156488,휴스틸,2019,579096387394.000000,27.084735,182538348103.000000
729,103176,흥국화재,2019,0.000000,0.000000,0.000000


In [5]:
all_19 = pd.read_csv('data/data_clean/2019(연간)/2019_연간보고서_상태표전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '자산총계', '부채총계']]
all_20 = pd.read_csv('data/data_clean/2020연간/2020_연간보고서_상태표전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '자산총계', '부채총계']]
all_21 = pd.read_csv('data/data_clean/2021연간/2021_연간보고서_상태표전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '자산총계', '부채총계']]
all_22 = pd.read_csv('data/data_clean/2022연간/2022_연간보고서_상태표전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '자산총계', '부채총계']]
all_23 = pd.read_csv('data/data_clean/2023연간/2023_연간보고서_상태표전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '자산총계', '부채총계']]
all_24 = pd.read_csv('data/data_clean/2024연간/2024_연간보고서_상태표전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '자산총계', '부채총계']]

In [6]:
ca_19 = adf_2019[(adf_2019['total_asset'] == 0) | (adf_2019['total_debt'] == 0)]
ca_20 = adf_2020[(adf_2020['total_asset'] == 0) | (adf_2020['total_debt'] == 0)]
ca_21 = adf_2021[(adf_2021['total_asset'] == 0) | (adf_2021['total_debt'] == 0)]
ca_22 = adf_2022[(adf_2022['total_asset'] == 0) | (adf_2022['total_debt'] == 0)]
ca_23 = adf_2023[(adf_2023['total_asset'] == 0) | (adf_2023['total_debt'] == 0)]
ca_24 = adf_2024[(adf_2024['total_asset'] == 0) | (adf_2024['total_debt'] == 0)]

In [7]:
print(
len(ca_19), len(ca_20), len(ca_21), len(ca_22), len(ca_23), len(ca_24)
)

54 43 44 43 1 1


In [8]:
ra_19 = all_19[all_19['회사명'].str.strip().isin(ca_19['corp_name'].str.strip())]
ra_20 = all_20[all_20['회사명'].str.strip().isin(ca_20['corp_name'].str.strip())]
ra_21 = all_21[all_21['회사명'].str.strip().isin(ca_21['corp_name'].str.strip())]
ra_22 = all_22[all_22['회사명'].str.strip().isin(ca_22['corp_name'].str.strip())]
ra_23 = all_23[all_23['회사명'].str.strip().isin(ca_23['corp_name'].str.strip())]
ra_24 = all_24[all_24['회사명'].str.strip().isin(ca_24['corp_name'].str.strip())]
ra_19

,회사명,자산총계,부채총계
243,기신정기,254063813641.000000,15366287167.000000
336,대동전자,135673856592.000000,21851934755.000000
458,동원금속,326377740111.000000,302703042404.000000
583,만호제강,245287788455.000000,18318870294.000000
862,세원정공,270396135360.000000,30942084566.000000
1235,오리엔트바이오,90656695306.000000,21298649061.000000
1426,이화산업,96877841923.000000,34756980820.000000
1601,체시스,83088662506.000000,57215227456.000000
1928,한국주철관공업,263831320833.000000,44108579235.000000
2054,형지엘리트,96304350489.000000,16631527885.000000


In [9]:
# 컬럼명 일치화
adf_columns = ['corp_name', 'total_asset', 'total_debt']
ra_19.columns = adf_columns
ra_20.columns = adf_columns
ra_21.columns = adf_columns
ra_22.columns = adf_columns
ra_23.columns = adf_columns
ra_24.columns = adf_columns
ra_19

,corp_name,total_asset,total_debt
243,기신정기,254063813641.000000,15366287167.000000
336,대동전자,135673856592.000000,21851934755.000000
458,동원금속,326377740111.000000,302703042404.000000
583,만호제강,245287788455.000000,18318870294.000000
862,세원정공,270396135360.000000,30942084566.000000
1235,오리엔트바이오,90656695306.000000,21298649061.000000
1426,이화산업,96877841923.000000,34756980820.000000
1601,체시스,83088662506.000000,57215227456.000000
1928,한국주철관공업,263831320833.000000,44108579235.000000
2054,형지엘리트,96304350489.000000,16631527885.000000


In [10]:
cadf_2019 = adf_2019.copy()
cadf_2020 = adf_2020.copy()
cadf_2021 = adf_2021.copy()
cadf_2022 = adf_2022.copy()
cadf_2023 = adf_2023.copy()
cadf_2024 = adf_2024.copy()

In [11]:
cadf_2019 = cadf_2019.merge(
    ra_19[['corp_name', 'total_asset', 'total_debt']],
    on='corp_name',
    how='left'
)
cadf_2019

,corp_code,corp_name,year,total_asset_x,ln_asset,total_debt_x,total_asset_y,total_debt_y
0,365387,AJ네트웍스,2019,1803255225986.000000,28.220615,1455914070366.000000,NaN,NaN
1,125080,AK홀딩스,2019,4327964555085.000000,29.096118,2898025666994.000000,NaN,NaN
2,219097,BGF,2019,1657381460248.000000,28.136260,129126735585.000000,NaN,NaN
3,1263022,BGF리테일,2019,2172410899706.000000,28.406859,1549815137631.000000,NaN,NaN
4,858364,BNK금융지주,2019,0.000000,0.000000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...
726,111421,휴니드테크놀러지스,2019,260965121649.000000,26.287653,145116828409.000000,NaN,NaN
727,362238,휴비스,2019,877170812678.000000,27.499968,481045043591.000000,NaN,NaN
728,156488,휴스틸,2019,579096387394.000000,27.084735,182538348103.000000,NaN,NaN
729,103176,흥국화재,2019,0.000000,0.000000,0.000000,NaN,NaN


In [12]:
mask_a = cadf_2019['total_asset_y'].notnull()
mask_d = cadf_2019['total_debt_y'].notnull()

In [13]:
cadf_2019.loc[mask_a, 'total_asset_x'] = cadf_2019.loc[mask_a, 'total_asset_y']
cadf_2019.loc[mask_d, 'total_debt_x'] = cadf_2019.loc[mask_d, 'total_debt_y']
cadf_2019[cadf_2019['corp_name'] == '기신정기']

,corp_code,corp_name,year,total_asset_x,ln_asset,total_debt_x,total_asset_y,total_debt_y
167,106614,기신정기,2019,254063813641.000000,0.000000,15366287167.000000,254063813641.000000,15366287167.000000


In [14]:
# 0 10개 감소
len(cadf_2019[(cadf_2019['total_asset_x'] == 0) | (cadf_2019['total_debt_x'] == 0)])

44

In [15]:
# 불필요 칼럼 제거, 칼럼명 변경
cadf_2019 = cadf_2019.drop(['total_asset_y', 'total_debt_y'], axis = 1)
cadf_columns = ['corp_code', 'corp_name', 'year', 'total_asset', 'asset_size', 'total_debt']

# 연도별 컬럼명 변경
cadf_2019.columns = cadf_columns
cadf_2020.columns = cadf_columns
cadf_2021.columns = cadf_columns
cadf_2022.columns = cadf_columns
cadf_2023.columns = cadf_columns
cadf_2024.columns = cadf_columns

In [16]:
cadf_2019

,corp_code,corp_name,year,total_asset,asset_size,total_debt
0,365387,AJ네트웍스,2019,1803255225986.000000,28.220615,1455914070366.000000
1,125080,AK홀딩스,2019,4327964555085.000000,29.096118,2898025666994.000000
2,219097,BGF,2019,1657381460248.000000,28.136260,129126735585.000000
3,1263022,BGF리테일,2019,2172410899706.000000,28.406859,1549815137631.000000
4,858364,BNK금융지주,2019,0.000000,0.000000,0.000000
...,...,...,...,...,...,...
726,111421,휴니드테크놀러지스,2019,260965121649.000000,26.287653,145116828409.000000
727,362238,휴비스,2019,877170812678.000000,27.499968,481045043591.000000
728,156488,휴스틸,2019,579096387394.000000,27.084735,182538348103.000000
729,103176,흥국화재,2019,0.000000,0.000000,0.000000


In [17]:
# log 씌우기
cadf_2019.loc[:, 'asset_size'] = np.where(cadf_2019['total_asset'] > 0, np.log(cadf_2019['total_asset']), 0)
cadf_2019['asset_size'].value_counts()

C:\Users\USER\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


asset_size
0.000000     44
28.220615     1
29.096118     1
28.136260     1
28.406859     1
             ..
26.994080     1
26.287653     1
27.499968     1
27.084735     1
26.789287     1
Name: count, Length: 688, dtype: int64

In [18]:
cadf_lst = [cadf_2019, cadf_2020, cadf_2021, cadf_2022, cadf_2023, cadf_2024]

for i in cadf_lst:
    i.loc[:, 'debt_ratio'] = np.where((i['total_asset'] > 0),
                               i['total_debt'] / i['total_asset'], 0)

In [19]:
cadf_2024

,corp_code,corp_name,year,total_asset,asset_size,total_debt,debt_ratio
0,365387,AJ네트웍스,2024,1717770434389.000000,28.172048,1274047255964.000000,0.741687
1,125080,AK홀딩스,2024,5336874928103.000000,29.305661,4091846427178.000000,0.766712
2,219097,BGF,2024,2300978258450.000000,28.464355,319875641992.000000,0.139017
3,1263022,BGF리테일,2024,3402590175982.000000,28.855558,2218457219442.000000,0.651991
4,858364,BNK금융지주,2024,152470490926534.000000,32.657992,141322916379933.000000,0.926887
...,...,...,...,...,...,...,...
726,111421,휴니드테크놀러지스,2024,241011444524.000000,26.208110,69226929333.000000,0.287235
727,362238,휴비스,2024,765377954903.000000,27.363636,530394278052.000000,0.692983
728,156488,휴스틸,2024,1431265670359.000000,27.989580,311625423766.000000,0.217727
729,103176,흥국화재,2024,12894689000000.000000,30.187837,12122030000000.000000,0.940079


In [20]:
year = range(2019, 2025)

for i, j in zip(year, cadf_lst):
    j.to_csv(f'data/Major_Shareholder/Major_Shareholder_correction/MSE_C_{i}.csv')